Name1: Peijie Yang
EID1: py2554 


Name2: Pengdi Xia
EID2: px353

# Association Analysis

Association analysis is one of the most used machine learning algorithms to extract hidden relationships from large datasets. In this assignment we'll be implementing two of the most commonly used algorithms for association rule mining - Apriori and FPTree algorithm.

The dataset (`large_retail.txt`) that we are going to use for this assignment has been adapted from the [Retail Market Basket Dataset](http://fimi.ua.ac.be/data/retail.pdf). The dataset contains transactions records supplied by an anonymous Belgian retail supermarket store. Each line in the file represents a separate transaction with the item ids separated by space. The dataset has 3000 transaction records and 99 different item ids.

We also provide a smaller dataset (`small_retail.txt`) with 9 transactions and 5 different item ids along with the solutions. *You should first test your implementation on this dataset, before running it on the larger dataset.*

The assignment will be **autograded**, we will use the `diff` command in linux to compare the output files. So please **check your answers** based on the given sample output files.

Implementation Hint:

- Use the `frozenset` data structure in Python (similar to `set` in functionality) to represent the itemsets because `frozenset` is a hashable data structure. You can just maintain a dictionary that maps from the itemset to its support count.

## Part 1 - Apriori Algorithm

Apriori algorithm is a classical algorithm in data mining. It is used for mining frequent itemsets and relevant association rules. In this part, you'll be implementing this algorithm for generating the itemsets that occur more than the `min_sup` threshold. Based on these frequent itemsets you'll find association rules that have confidence above the `min_conf` threshold.

In [44]:
# Standard imports (you can add additional headers if you wish)
import numpy

In [45]:
# Reading the dataset from file
def load_dataset(file_name):
    with open(file_name, 'r') as f:
        content = f.readlines()
        data = [[int(x) for x in line.rstrip().split()] for line in content]
    return data

In [46]:
small_dataset = load_dataset('small_retail.txt')
small_dataset

[[1, 2, 5],
 [2, 4],
 [2, 3],
 [1, 2, 4],
 [1, 3],
 [2, 3],
 [1, 3],
 [1, 2, 3, 5],
 [1, 2, 3]]

**Q1.** Implement the function `createSet1` that takes input the entire dataset and returns the list of the set of size 1 itemsets. For example, for `small_retail.txt` it should return:
~~~
[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]
 ~~~
 Please **don't hardcode** the item ids, your code should support item ids that are non-sequential.

In [47]:
def createSet1(dataset):
    c1 = []
    # your code goes here
    temp = []
    for i in range(len(dataset)):
        for j in range(len(dataset[i])):
            temp.append(dataset[i][j])
    for k in set(temp):
        temp2 =[]
        temp2.append(k)
        c1.append(frozenset(temp2))
    return c1

In [48]:
createSet1(small_dataset)

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

**Q2.** Implement function `filter_candidates` that takes input the candidate itemsets, dataset and the minumum support count `min_sup` and filters out candidates that don't meet the support threshold.

**Hint:** You should also return the support count information (perhaps as a `dict`) for the itemsets. This will be useful later on for the final output.

In [49]:
def filter_candidates(candidate, dataset, min_sup):
    retlist = []
    support_data = []
    indicate = True
    # your code goes here
    for i in range(len(candidate)):
        count = 0
        temp = []
        for j in range(len(dataset)):
            # subset operation
            if set(candidate[i]) <= set(dataset[j]):
                count += 1
        if count >= min_sup:
            retlist.append(candidate[i])
            temp.append(candidate[i]) 
            temp.append(count)
            support_data.append(temp)
    return retlist, support_data

In [50]:
freq1, sup1 = filter_candidates(createSet1(small_dataset), small_dataset, 2)
print (freq1, sup1)

[frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5})] [[frozenset({1}), 6], [frozenset({2}), 7], [frozenset({3}), 6], [frozenset({4}), 2], [frozenset({5}), 2]]


**Q3.** Implement the function `generateNextItemsets` that takes in itemsets of size `k` and generates candidate itemsets of size `k + 1`.

**Hint:** Use the fact that if `[1, 2, 3, 4]` is a frequent itemset of size 4 then `[1, 2, 3]` and `[1, 2, 4]` both will be frequent itemsets of size 3. You can use this to reduce the number of candidate itemsets that you need to check drastically.

In [51]:
def generateNextItemsets(freq_sets):
    retlist = []
    # your code goes here
    # find the last item
    temp = []
    for i in range(len(freq_sets)):
        temp.append(max(list(freq_sets[i])))
    # remove the duplicate value
    temp2 = []
    for m in set(temp):
        temp2.append(m)
    # compare part
    for j in range(len(freq_sets)):
        for k in range(len(temp2)):
            if max(list(freq_sets[j])) < temp2[k]:
                templist = list(freq_sets[j])
                templist.append(temp2[k])
                retlist.append(frozenset(templist))
    return retlist

In [52]:
generateNextItemsets(freq1)

[frozenset({1, 2}),
 frozenset({1, 3}),
 frozenset({1, 4}),
 frozenset({1, 5}),
 frozenset({2, 3}),
 frozenset({2, 4}),
 frozenset({2, 5}),
 frozenset({3, 4}),
 frozenset({3, 5}),
 frozenset({4, 5})]

**Q4.** Implement the function `aprioriFreqItemsets` that takes the entire dataset as the input and returns the frequent itemsets that have support count more than `min_sup`.

In [53]:
def aprioriFreqItemsets(dataset, min_sup):
#     # your code goes here
    retlist = []
    supp_data =[]
    init_set = createSet1(dataset)
    init_retlist, init_supp = filter_candidates(init_set, dataset, min_sup)
    retlist +=init_retlist
    supp_data +=init_supp
    templist = generateNextItemsets(init_retlist)
    while templist != []:
        templist, temp_data = filter_candidates(templist, dataset, min_sup)
        if templist != []: 
            retlist +=templist
            supp_data +=temp_data
        templist = generateNextItemsets(templist)
    return retlist, supp_data

In [54]:
retlist,supp_data= aprioriFreqItemsets(small_dataset,2)


**Q5.** Display the frequent item sets in the form of a table along with their `support_fraction` for the `large_retail.txt` dataset with min support count 300.

Sample Table Format (tab separated table)

~~~
Sup     Freq Itemset
0.67	[1]
0.44	[1, 2]
(and so on)
...
...
~~~

`support_fraction(itemset) = support_count(itemset) / num_total_transactions`.

Note that the `support_fraction` should be rounded to the nearest 2 decimal places (use `round(sup, 2)`). Also `support_fraction` and the itemset should be separated by a tab (`'\t'`). The itemsets should also be in a sorted order where smaller itemsets should come before larger itemsets and itemsets of the same size should be sorted amongst themselves.

For eg. 
~~~~
[1, 2] should come before [1, 2, 3]
[1, 2, 3] should come before [1, 2, 4]
[1, 2, 3] should come before [1, 4, 5]
[1, 2, 3] should come before [2, 3, 4]
~~~~

Note that **this order is very important** because your output will be checked using the `diff` command. The output also **shouldn't contain any duplicates**. The sample output for the `small_retail.txt` dataset with min support count as 2 is:

~~~~
Sup     Freq Itemset
0.67	[1]
0.78	[2]
0.67	[3]
0.22	[4]
0.22	[5]
0.44	[1, 2]
0.44	[1, 3]
0.22	[1, 5]
0.44	[2, 3]
0.22	[2, 4]
0.22	[2, 5]
0.22	[1, 2, 3]
0.22	[1, 2, 5]
~~~~

**Store** this output for the `large_retail.txt` dataset in the file `apriori_itemsets.txt`. The sample output file for the `small_retail.txt` dataset has been provided to you as `small_apriori_itemsets.txt` for your convenience.

In [55]:
# your code goes here
large_dataset = load_dataset('large_retail.txt')
retlist,retmap = aprioriFreqItemsets(large_dataset,300)
file1 = open("apriori_itemsets.txt","w") 
file1.write("Sup\tFreq Itemset")  
for i in range(len(retmap)):
    file1.write("\n"+str(round(retmap[i][1]/len(large_dataset),2))+"\t"+str(sorted(list(retmap[i][0]), key=int)))   
file1.close()

**Q6.** Display the closed frequent item sets along with their `support_fraction` in the same format as specified in Q5.

In [56]:
# your code goes here
temp_list = []
for i in range(len(retlist)):
    temp_list.append(retlist[i])
print(retmap)
print("Sup\tFreq Itemset")
for i in range(len(temp_list)):
    write = True
    for j in range(len(temp_list)):
        if temp_list[i] < temp_list[j] and retmap[i][1] == retmap[j][1]:
            write = False
    if write:
        print(str(round(retmap[i][1]/len(large_dataset),2))+"\t"+str(sorted(list(retmap[i][0]), key=int)))

[[frozenset({31}), 309], [frozenset({32}), 691], [frozenset({36}), 320], [frozenset({38}), 771], [frozenset({39}), 1793], [frozenset({41}), 938], [frozenset({48}), 1396], [frozenset({60}), 337], [frozenset({65}), 329], [frozenset({89}), 343], [frozenset({32, 39}), 420], [frozenset({32, 48}), 370], [frozenset({38, 39}), 520], [frozenset({41, 38}), 323], [frozenset({48, 38}), 384], [frozenset({41, 39}), 697], [frozenset({48, 39}), 982], [frozenset({48, 41}), 534], [frozenset({48, 41, 39}), 428]]
Sup	Freq Itemset
0.1	[31]
0.23	[32]
0.11	[36]
0.26	[38]
0.6	[39]
0.31	[41]
0.47	[48]
0.11	[60]
0.11	[65]
0.11	[89]
0.14	[32, 39]
0.12	[32, 48]
0.17	[38, 39]
0.11	[38, 41]
0.13	[38, 48]
0.23	[39, 41]
0.33	[39, 48]
0.18	[41, 48]
0.14	[39, 41, 48]


**Q. (EXTRA CREDIT +5 points)** Generate the rules having confidence above `min_conf = 0.5` using the frequent itemsets generated in Q5. Display the rules in the form of a table.

Sample table format (tab separated table)

~~~
Sup     Conf    Rule
0.44	0.67	[1] -> [2]
0.22	1.0	 [5] -> [1, 2]
0.22	1.0	 [2, 5] -> [1]
(and so on)
...
...
~~~

Note that rule confidence should be rounded to the nearest 2 decimal places (use `round(conf, 2)`). This table should also be tab (`'\t'`) separated. The rules should be displayed in the sorted order. If a rule is given as `LHS -> RHS` then the rules for which `len(LHS)` is lesser should appear first. If the `len(LHS)` is equal for two rules then rules for which `len(RHS)` is lesser should appear first. If both `len(LHS)` and `len(RHS)` is equal then the rules should be sorted based on LHS first and then based on RHS.

~~~~
Note:
LHS (Left Hand Side)
RHS (Right Hand Side)
~~~~

For eg.
~~~~
[3] -> [2] should come before [1, 3] -> [4]
[4] -> [2] should come before [2] -> [3, 4]
[1, 3] -> [2] should come before [1, 5] -> [2]
[1, 2] -> [3] should come before [1, 2] -> [5]
~~~~

Note that **this order is very important** because your output will be checked using the `diff` command. The sample output for the `small_retail.txt` dataset with `min_conf = 0.5` is:

~~~~
Sup	 Conf	Rule
0.44	0.67	[1] -> [2]
0.44	0.67	[1] -> [3]
0.44	0.57	[2] -> [1]
0.44	0.57	[2] -> [3]
0.44	0.67	[3] -> [1]
0.44	0.67	[3] -> [2]
0.22	1.0	 [4] -> [2]
0.22	1.0	 [5] -> [1]
0.22	1.0	 [5] -> [2]
0.22	1.0	 [5] -> [1, 2]
0.22	0.5	 [1, 2] -> [3]
0.22	0.5	 [1, 2] -> [5]
0.22	0.5	 [1, 3] -> [2]
0.22	1.0	 [1, 5] -> [2]
0.22	0.5	 [2, 3] -> [1]
0.22	1.0	 [2, 5] -> [1]
~~~~

**Store** this output for the `large_retail.txt` dataset in the file `apriori_rules.txt`. The sample output file for the `small_retail.txt` dataset has been provided to you as `small_apriori_rules.txt` for your convenience.

**Hint:** you don't actually need to traverse the entire dataset to compute the confidence for a rule since you have already computed the `support_data` for all the frequent itemsets. `conf(LHS -> RHS) = sup(LHS union RHS) / sup(LHS)`.

In [57]:
# your code goes here
# build a dictrionary base on sup_data
def dic (supp_data):
    result = {}
    for i in range(len(supp_data)):
        result[supp_data[i][0]] = supp_data[i][1]
    return result

In [58]:
retlist,sup_data = aprioriFreqItemsets(large_dataset, 300)
sup_data

[[frozenset({31}), 309],
 [frozenset({32}), 691],
 [frozenset({36}), 320],
 [frozenset({38}), 771],
 [frozenset({39}), 1793],
 [frozenset({41}), 938],
 [frozenset({48}), 1396],
 [frozenset({60}), 337],
 [frozenset({65}), 329],
 [frozenset({89}), 343],
 [frozenset({32, 39}), 420],
 [frozenset({32, 48}), 370],
 [frozenset({38, 39}), 520],
 [frozenset({38, 41}), 323],
 [frozenset({38, 48}), 384],
 [frozenset({39, 41}), 697],
 [frozenset({39, 48}), 982],
 [frozenset({41, 48}), 534],
 [frozenset({39, 41, 48}), 428]]

In [59]:
from operator import itemgetter
# your code goes here
result = []
dictionary = dic(sup_data)
element = []
# loop over the freq itemset
for i in range(len(sup_data)):
    temp = []
    lhs = []
    if len(sup_data[i][0]) == 2 :
        temp.append(i)
        lhs.append(sorted(list(sup_data[i][0]), key=int)[0])
        temp.append(lhs)
        temp.append(list(set(sup_data[i][0])-set(lhs)))
        element.append(temp) 
    elif len(sup_data[i][0]) == 3 :
        for j in range(len(list(sup_data[i][0]))):
            lhs = []
            temp = []
            temp.append(i)
            lhs.append(sorted(list(sup_data[i][0]), key=int)[j])
            temp.append(lhs)
            temp.append(sorted(list(set(sup_data[i][0])-set(lhs)), key=int))
            element.append(temp)
# fill it up
length = len(element)
for k in range(length):
    temp = []
    lhs = element[k][2]
    rhs = element[k][1]
    temp.append(element[k][0])
    temp.append(lhs)
    temp.append(rhs)
    element.append(temp)
    
# fliter
for k in range(len(element)):
    conf=round(sup_data[element[k][0]][1]/dictionary[frozenset(element[k][1])],2)
    if conf >=0.5:
        element[k].append(conf)
        result.append(element[k])
result = sorted(result, key=itemgetter(1,2))
result = sorted(result, key= lambda result: len(result[1]))
#write into file
file2 = open("apriori_rules.txt","w") 
file2.write("Sup\tConf\tRule")  
for i in range(len(result)):
    file2.write("\n"+str(round(sup_data[result[i][0]][1]/len(large_dataset),2))
                +"\t"+str(result[i][3])
                +"\t"+str(result[i][1]) +" -> "+str(result[i][2]))
file2.close()

## Part 2 - Frequent Pattern Tree

The FP-Growth Algorithm, proposed by [Han](https://www.cs.sfu.ca/~jpei/publications/sigmod00.pdf), is an efficient and scalable method for mining the complete set of frequent patterns by pattern fragment growth, using an extended prefix-tree structure for storing compressed and crucial information about frequent patterns named frequent-pattern tree (FP-tree). [wiki]

In [60]:
# variables:
# self.item_id: item id of the item
# self.item_count: item count for this node
# self.node_link: next pointer for the linked list that connects
#                 nodes of the same item_id (required for the FP-growth algorithm)
# self.parent: pointer to the parent node (should be None for the root)
# self.children: dictionary for the children (maps from item_id to the FPTreeNode object)
# NOTE: root node should have item_id as -1

class FPTreeNode:
    def __init__(self, uid, num):
        self.item_id = uid
        self.item_count = num
        self.node_link = None
        self.parent = None
        self.children = {}
    def displayTree(self, tab=1):
        if self.item_id == -1:
            print ('  '*tab, 'root')
        else:
            print ('  '*tab, 'item_id:', self.item_id, 'item_count:', self.item_count)
        for key in sorted(self.children.keys()):
            self.children[key].displayTree(tab + 1)
    # helper function for saveToFile
    def saveToFile_helper(self, fp, tab=1):
        if self.item_id == -1:
            print ('  '*tab, 'root', file=fp)
        else:
            print ('  '*tab, 'item_id:', self.item_id, 'item_count:', self.item_count, file=fp)
        for key in sorted(self.children.keys()):
            self.children[key].saveToFile_helper(fp, tab + 1)
    # call this to save to file
    def saveToFile(self, filename):
        with open(filename, 'w') as fp:
            self.saveToFile_helper(fp)

**Q7.** Build the FP-Tree for the `large_retail.txt` dataset with minimum support count as 300 and display the tree using the `displayTree` function. Also save this output to `fp_tree.txt` using the `saveToFile` function.

Note that while reordering the transactions based on their frequency, **item_ids that have equal frequency should be ordered based on the item_id value**.

For eg.
~~~~
For the small_retail.txt dataset:
{item_id: frequency} -> {1: 6, 2: 7, 3: 6, 4: 2, 5: 2}
The transaction [1, 2, 3, 5] should be reordered as [2, 1, 3, 5]
Notice the relative ordering of 1 and 3 (both have 6 occurences in the dataset)
~~~~

Note that the **relative order between transactions should not be changed**, they should be inserted in the same order as they appear in the dataset. For the `small_retail.txt` the transactions would be inserted in the FP Tree in this order:
~~~~
[2, 1, 5]
[2, 4]
[2, 3]
[2, 1, 4]
[1, 3]
[2, 3]
[1, 3]
[2, 1, 3, 5]
[2, 1, 3]
~~~~

The tree output for `small_retail.txt` dataset is given as follows:
~~~~
   root
     item_id: 1 item_count: 2
       item_id: 3 item_count: 2
     item_id: 2 item_count: 7
       item_id: 1 item_count: 4
         item_id: 3 item_count: 2
           item_id: 5 item_count: 1
         item_id: 4 item_count: 1
         item_id: 5 item_count: 1
       item_id: 3 item_count: 2
       item_id: 4 item_count: 1
~~~~
This output has been provided to you as `small_fp_tree.txt` for your convenience. You can use the `diff` command in Linux to check your output with the provided output.

In [61]:
# your code goes here
import operator
min_sup = 300
small_dataset = load_dataset('large_retail.txt')
frequency_set = {}

for x in small_dataset: 
    for y in x:
        if y not in frequency_set:
            frequency_set[y] = 1
        else: 
            frequency_set[y] += 1
sorted_set = sorted(frequency_set.items(), key=lambda x: (x[1] ,-x[0]), reverse=True)
#sorted_set = {2: 7, 1: 6, 3: 6, 4: 2, 5: 2}
#put sorted keys into sort_set = {2,1,3,4,5}
sort_set = []
for x in sorted_set:
    if x[1] >= min_sup:
        sort_set.append(x[0])
new_small_set = []
for x in small_dataset:
    temp = []
    for y in x:
        if y in sort_set:
            temp.append(y)
    if len(temp) != 0:
        new_small_set.append(temp)

new_freq_set={}
count = len(sort_set)
for x in sort_set:
    new_freq_set[x] = count
    count -= 1

new_data_set = []
for x in new_small_set:
    temp = {}
    new_set = []
    for y in x:
        temp[y] = new_freq_set[y]
    #sorted by weight assigned in new_freq_set
    temp2 = sorted(temp.items(), key=operator.itemgetter(1),reverse=True)
    for z in temp2:
        new_set.append(z[0])
    new_data_set.append(new_set)



def leaf(uid,  parent_node, count):
    leafNode = FPTreeNode(-1,1)
    leafNode.item_id = uid
    leafNode.item_count = count
    leafNode.parent = parent_node
    leafNode.children = {}
    return leafNode

def sublist (data):
    sublist_result = {}
    uid_dict = {}
    for x in data: 
        if x[0] in sublist_result:
            if len(x) > 1:
                sublist_result[x[0]].append(x[1:])
        else:
            sublist_result[x[0]] = [x[1:]]
        if x[0] in uid_dict:
            uid_dict[x[0]] += 1
        else:
            uid_dict[x[0]] = 1
    for i in sublist_result:
        for j in sublist_result[i]:
            if len(j) == 0 and len(sublist_result[i]) != 1:
                sublist_result[i].remove(j)
    return sublist_result, uid_dict

firstNode = {} 
nodeLink = {}
def createTree(data, parent_node,firstNode,nodeLink):
    sublist_result, uid_dict = sublist (data)
    for x in sublist_result:
        node = FPTreeNode(x,0)
        if len(sublist_result[x][0]) == 0:
            node = leaf(x, parent_node, uid_dict[x])
            parent_node.children[x] = node
        else:
            node = FPTreeNode(x,uid_dict[x]) 
            node.parent = parent_node
            parent_node.children[x] = node
            createTree(sublist_result[x], node,firstNode,nodeLink)
        if x not in firstNode:
            firstNode[x] = node
            nodeLink[x] = node
        else:
            nodeLink[x].node_link = node
            nodeLink[x] = nodeLink[x].node_link
    return parent_node

myTree = createTree(new_data_set, FPTreeNode(-1,1),firstNode,nodeLink)
# myTree.displayTree()
myTree.saveToFile('fp_tree.txt')

Q8. Implement the FP-growth algorithm to generate the frequent itemsets from the FP-tree you generate in Q7 (with min support count of 300). Display the frequent item sets in the same format as specified in Q5. Store this output for the large_retail.txt dataset in the file fp_growth_itemsets.txt. Remember you are only allowed to mine the FP tree that you generate in Q7. you cannot use the dataset as the input to your algorithm.

In [62]:
def FPGrowth(root, cur, minsup):    
    if root.item_id != -1:
        cur.append(root.item_id)
        first_node = firstNode[root.item_id]
        count = 0
        while first_node != None:
            temp2 = first_node
            temp = []
            while temp2.item_id != -1:
                temp.append(temp2.item_id)
                temp2 = temp2.parent
            if set(cur).issubset(set(temp)): 
                count += first_node.item_count
            first_node = first_node.node_link
        if count >= minsup:
            key = frozenset(cur)
            freq_itemset[key] = count        
    for child in root.children.values():
        FPGrowth(child, cur, minsup) 
    if root.item_id != -1:
        cur.remove(root.item_id)
            

freq_itemset = {}
FPGrowth(myTree, [], 300)
temp_list = []
retmap = []

file4 = open('fp_growth_itemsets.txt', 'w')
freq_dict = {}
for x in freq_itemset:
    temp_list.append(x)
    retmap.append([x,freq_itemset[x]])
# print(retmap)
freq_sorted_list = []
for x in range(len(temp_list)):
    freq_sorted_list.append(((round(retmap[x][1]/len(small_dataset),2)),sorted(list(retmap[x][0]), key=int)))
# print(freq_sorted_list)

sorted_set = sorted(freq_sorted_list, key=lambda x: (len(x[1]), x[1]), reverse=False)
file4.write('Sup\tFreq Itemset')
for x in sorted_set:
    file4.write('\n' + str(x[0]) + "\t"+str(x[1]))
file4.close()